We cannot work with the text data in machine learning so we need to convert them into numerical vectors, As a part of this practice exercise you will implement different techniques to do the same.

In this notebook we are going to understand some basic text cleaning steps and techniques for encoding text data. We are going to learn about
1. **Understanding the data** - See what's data is all about. what should be considered for cleaning for data (Punctuations , stopwords etc..).
2. **Basic Cleaning** -We will see what parameters need to be considered for cleaning of data (like Punctuations , stopwords etc..)  and its code.
3. **Techniques for Encoding** - All the popular techniques that are used for encoding that I personally came across.
    *           **Bag of Words**
    *           **Binary Bag of Words**
    *           **Bigram, Ngram**
    *           **TF-IDF**( **T**erm  **F**requency - **I**nverse **D**ocument **F**requency)


# 1.Importing Libraries

Libraries used in this notebook along with their version:

google	2.0.3

nltk	3.2.5

numpy	1.18.3

pandas	1.0.3

In [1]:
import nltk
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2.Reading the data

*We* will employ a text categorization dataset based on Reviews. Each article is assigned a specific captegory. 
###Implement the code to load the dataset.(Hint: Use the pandas library to load the csv file.)

In [7]:
data = pd.read_csv("/content/drive/MyDrive/NLP/Session1/Reviews.csv", index_col=0)
print(data.shape)
data.head()

(78720, 9)


,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
Id,,,,,,,,,
1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


1. **Understanding the data**

Our main objective from the dataset is to predict whether a review is **Positive** or **Negative** based on the Text.
 
If we see the Score column, it has values 1,2,3,4,5 .  Considering 1, 2 as Negative reviews and 4, 5 as Positive reviews.
 For Score = 3 we will consider it as Neutral review and lets delete the rows that are neutral, so that we can predict either Positive or Negative
 
HelpfulnessNumerator says about number of people found that review usefull and HelpfulnessDenominator is about usefull review count + not so usefull count.
So, from this we can see that HelfulnessNumerator is always less than or equal to HelpfulnesDenominator.

In [8]:
data.drop(data[data.Score == 3].index, inplace=True)

Converting Score values into class label either Positive or Negative.

In [9]:
data["label"] = data.Score.map({4:'Positive',5:'Positive',1:'Negative' , 2:'Negative'})

In [10]:
data.sample(3)

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,label
Id,,,,,,,,,,
9828,B001EQ5CK2,APIIRLP1JR2VI,spunky_m0nkey,0,0,5,1295913600,"Great coffee, great price!",This has become our new coffee of choice for t...,Positive
16428,B007TJGZ54,A3V0B6OPOQZNDK,"Elisa Veneziano ""EV""",0,0,4,1351036800,Morning coffee,I use this as my morning coffee wake up call. ...,Positive
43392,B0038QXXNO,A1N7XTAFCDJEU7,Cynthia A,2,4,5,1318896000,Cheese Heads I love you!,I love Cheese Heads and I love Wisconsin White...,Positive


2. **Basic Cleaning**
 
**Deduplication** means removing duplicate rows, It is necessary to remove duplicates in order to get unbaised results. Checking duplicates based on UserId, ProfileName, Time, Text. If all these values are equal then we will remove those records. (No user can type a review on same exact time for different products.)


We have seen that HelpfulnessNumerator should always be less than or equal to HelpfulnessDenominator so checking this condition and removing those records also.


In [11]:
data[data.duplicated(subset='UserId')]

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,label
Id,,,,,,,,,,
30,B0001PB9FY,A3HDKO7OW0QNK4,Canadian Fan,1,1,5,1107820800,The Best Hot Sauce in the World,I don't know if it's the cactus or the tequila...,Positive
241,B001EO5ZMY,AF72GTWZGAC61,Pinkhat,3,3,5,1330732800,Excellent loose tea.,"If you like the Ceylon tea variety, you will c...",Positive
427,B000G6RYNE,A1Y3XPZK9ZADFW,albinocrow,0,0,4,1334016000,"pretty good, could be better",Glad to find these in a one ounce size but the...,Positive
430,B000G6RYNE,A1IRN1M05TPOVT,"Sharon M. Helfand ""Scrapper""",0,0,5,1331078400,Kettle potato chips: Sweet onion,WOW! I have eaten quite a few potato chips in...,Positive
436,B000G6RYNE,A15USNEAJUXOSH,L. Schrank,0,0,5,1326067200,Delicious,"I love these chips, I buy the 24 pack once a m...",Positive
...,...,...,...,...,...,...,...,...,...,...
78716,B00472I5A4,A1JBB01LAK69LV,Rubyred,1,1,4,1212105600,The chip with a kiss of salt,These chips are delectably delicious. I am on ...,Positive
78717,B00472I5A4,AKMEY1BSHSDG7,J. Arena,1,1,5,1202774400,Crunch. Wow!,"Not too salty like regular supermarket chips, ...",Positive
78718,B00472I5A4,A2F7GUZ4UMVHFL,James Walters,1,1,5,1186876800,Great strong flavor,"I agree with the company motto ""A Natural Obse...",Positive


Converting all words to lowercase and removing punctuations and html tags if any

**Stemming**- Converting the words into their base word or stem word ( Ex - tastefully, tasty,  these words are converted to stem word called 'tasti'). This reduces the vector dimension because we dont consider all similar words  

**Stopwords** - Stopwords are the unnecessary words that even if they are removed the sentiment of the sentence dosent change.

Ex -    This pasta is so tasty ==> pasta tasty    ( This , is, so are stopwords so they are removed)

To see all the stopwords see the below code cell.

###Create a function called "complaint_to_words" to convert each consumer complaint narrative to individual tokens.(Hint: Use regular expression based tokenizer.)

In [12]:
complaint = data["Text"]
complaint

Id
1        I have bought several of the Vitality canned d...
2        Product arrived labeled as Jumbo Salted Peanut...
3        This is a confection that has been around a fe...
4        If you are looking for the secret ingredient i...
5        Great taffy at a great price.  There was a wid...
                               ...                        
78716    These chips are delectably delicious. I am on ...
78717    Not too salty like regular supermarket chips, ...
78718    I agree with the company motto "A Natural Obse...
78719    The New York Cheddar flavor is my favorite of ...
78720                                                  NaN
Name: Text, Length: 72322, dtype: object

In [13]:
complaint = complaint[:1000]

# 3.Basic Cleaning

We will use the above function here to create a list of list that will store each complaint tokenized into separate words.

## 3.1.Tokenize

In [14]:
from six import string_types
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize

nltk.download('stopwords') # Downloading stopwords
nltk.download('punkt') # Downloading tokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
word = []
for i in complaint:
  words = word_tokenize(i)
  word.extend(words)

len(word)

87728

In [16]:
print(word[:100])

['I', 'have', 'bought', 'several', 'of', 'the', 'Vitality', 'canned', 'dog', 'food', 'products', 'and', 'have', 'found', 'them', 'all', 'to', 'be', 'of', 'good', 'quality', '.', 'The', 'product', 'looks', 'more', 'like', 'a', 'stew', 'than', 'a', 'processed', 'meat', 'and', 'it', 'smells', 'better', '.', 'My', 'Labrador', 'is', 'finicky', 'and', 'she', 'appreciates', 'this', 'product', 'better', 'than', 'most', '.', 'Product', 'arrived', 'labeled', 'as', 'Jumbo', 'Salted', 'Peanuts', '...', 'the', 'peanuts', 'were', 'actually', 'small', 'sized', 'unsalted', '.', 'Not', 'sure', 'if', 'this', 'was', 'an', 'error', 'or', 'if', 'the', 'vendor', 'intended', 'to', 'represent', 'the', 'product', 'as', '``', 'Jumbo', "''", '.', 'This', 'is', 'a', 'confection', 'that', 'has', 'been', 'around', 'a', 'few', 'centuries', '.']


## 3.2.Lower Case

In [17]:
tokens  = [w.lower() for w in word]

## 3.3.Removing Stopwords

### 3.3.1.Removing Punctuation

In [18]:
punch = ["''","``","...","-",".",";",":","!","?","/","\\",",","#","@","$","&",")","(","\""]

filter_tokens = [w for w in tokens if not w in punch]

print(filter_tokens[:100])

['i', 'have', 'bought', 'several', 'of', 'the', 'vitality', 'canned', 'dog', 'food', 'products', 'and', 'have', 'found', 'them', 'all', 'to', 'be', 'of', 'good', 'quality', 'the', 'product', 'looks', 'more', 'like', 'a', 'stew', 'than', 'a', 'processed', 'meat', 'and', 'it', 'smells', 'better', 'my', 'labrador', 'is', 'finicky', 'and', 'she', 'appreciates', 'this', 'product', 'better', 'than', 'most', 'product', 'arrived', 'labeled', 'as', 'jumbo', 'salted', 'peanuts', 'the', 'peanuts', 'were', 'actually', 'small', 'sized', 'unsalted', 'not', 'sure', 'if', 'this', 'was', 'an', 'error', 'or', 'if', 'the', 'vendor', 'intended', 'to', 'represent', 'the', 'product', 'as', 'jumbo', 'this', 'is', 'a', 'confection', 'that', 'has', 'been', 'around', 'a', 'few', 'centuries', 'it', 'is', 'a', 'light', 'pillowy', 'citrus', 'gelatin', 'with', 'nuts']


### 3.3.2.Removing the Stop Words

In [19]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
filter_token1 = [w for w in filter_tokens if not w in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 3.4.Stemming & Lemitization

### 3.4.1.Stemming

In [20]:
from nltk.stem import PorterStemmer

port = PorterStemmer()

sent = list(map(lambda x: port.stem(x), filter_token1))

### 3.4.2.Lemitization

In [21]:
from nltk.stem import LancasterStemmer
lancaster=LancasterStemmer()

sentence = list(map(lambda x : lancaster.stem(x),sent))

## 3.5.PoS

In [22]:
nltk.download('averaged_perceptron_tagger')
pos = nltk.pos_tag(sentence)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Save the data

In [23]:
posdf = pd.DataFrame(pos)
posdf.head()

,0,1
0,bought,VBN
1,sev,NNS
2,vit,NN
3,can,MD
4,dog,VB


# 4.**Techniques for Encoding**

4. **Techniques for Encoding**

      **BAG OF WORDS**
      
      In BoW we construct a dictionary that contains set of all unique words from our text review dataset.The frequency of the word is counted here. if there are **d** unique words in our dictionary then for every sentence or review the vector will be of length **d** and count of word from review is stored at its particular location in vector. The vector will be highly sparse in such case.
      
      Ex. pasta is tasty and pasta is good
      
     **[0]....[1]............[1]...........[2]..........[2]............[1]..........**             <== Its vector representation ( remaining all dots will be represented as zeroes)
     
     **[a]..[and].....[good].......[is].......[pasta]....[tasty].......**            <==This is dictionary
      .
      
    Using scikit-learn's CountVectorizer we can get the BoW and check out all the parameters it consists of, one of them is max_features =5000 it tells about to consider only top 5000 most frequently repeated words to place in a dictionary. so our dictionary length or vector length will be only 5000
    


   **BINARY BAG OF WORDS**
    
   In binary BoW, we dont count the frequency of word, we just place **1** if the word appears in the review or else **0**. In CountVectorizer there is a parameter **binary = true** this makes our BoW to binary BoW.
   
  

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
toNumeric = CountVectorizer(binary = True)

In [25]:
toNumeric.fit(posdf[0])
X_train_dtm = toNumeric.transform(posdf[0])

In [26]:
X_train_dtm

<41723x4223 sparse matrix of type '<class 'numpy.int64'>'
	with 38297 stored elements in Compressed Sparse Row format>

In [27]:
X_train_dtm.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

 **Drawbacks of BoW/ Binary BoW**
 
 Our main objective in doing these text to vector encodings is that similar meaning text vectors should be close to each other, but in some cases this may not possible for Bow
 
For example, if we consider two reviews **This pasta is very tasty** and **This pasta is not tasty** after stopwords removal both sentences will be converted to **pasta tasty** so both giving exact same meaning.

The main problem is here we are not considering the front and back words related to every word, here comes Bigram and Ngram techniques.

## 4.1.N-gram 

### 4.1.1.**BI-GRAM BOW**

Considering pair of words for creating dictionary is Bi-Gram , Tri-Gram means three consecutive words so as NGram.

CountVectorizer has a parameter **ngram_range** if assigned to (1,2) it considers Bi-Gram BoW

But this massively increases our dictionary size 

In [28]:
toNumeric = CountVectorizer(binary = True, ngram_range=(1,2))
toNumeric.fit(posdf[0])
X_train_dtm = toNumeric.transform(posdf[0])
X_train_dtm.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## 4.2.**TF-IDF**

**Term Frequency -  Inverse Document Frequency** it makes sure that less importance is given to most frequent words and also considers less frequent words.

**Term Frequency** is number of times a **particular word(W)** occurs in a review divided by totall number of words **(Wr)** in review. The term frequency value ranges from 0 to 1.

**Inverse Document Frequency** is calculated as **log(Total Number of Docs(N) / Number of Docs which contains particular word(n))**. Here Docs referred as Reviews.


**TF-IDF** is **TF * IDF** that is **(W/Wr)*LOG(N/n)**


 Using scikit-learn's tfidfVectorizer we can get the TF-IDF.

So even here we get a TF-IDF value for every word and in some cases it may consider different meaning reviews as similar after stopwords removal. so to over come we can use BI-Gram or NGram.

In [29]:
def tf_idf(word, token):
    if word not in word_index:
        return .0
    return word_tf(word, token) * word_idf[word_index[word]]

## 4.3.Word2Vec

Gensim is a free to use python library. It provides APIs to solve various problems relating to natural language processing. It is fast, scalable and robust.

In this practice exercise we will train our own Word2Vec model using gensim Word2Vec API. Objectives of this practice exercise are, 


1.   Train your word2vec word embedding model.
2.   Visualize trained word embedding model using principal component analysis.


First step will be to load the corpus, clean it and tokenize it.

Libraries used in this notebook along with their version:

google	2.0.3

matplotlib	3.2.1

numpy	1.18.3

pandas	1.0.3

Next step is to import the Word2Vec model from gensim.

##### Create your own model using the data_list defined above and gensim Word2Vec API. (Hint: https://radimrehurek.com/gensim/models/word2vec.html)

##### Use PCA algorithm from sklearn to convert high dimesnional word embeddings to two diemnsions and save them in the variable "results".

##### Visualizing the word embeddings.

# 5.Emotion and Sentiment Analysis